In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [7]:
spark = (
    SparkSession.builder.appName('Curso PySpark')
    .config('spark.sql.repl.eagerEval.enabled',True)
    .getOrCreate()
)

In [9]:
spark

In [16]:
df = spark.read.parquet('C:/Users/Kimura/Desktop/Curso PySpark/DATASETS/DATASETS/LOGINS.parquet')

In [22]:
# Aqui basicamente criei outro DataFrame para adicionar as colunas que serão utilizadas nesse notebook.
# O DataFrame original ainda contêm todos os dados.

df_01 = df.select('email','senha','estado','cor_favorita','profissao')
df_01

email,senha,estado,cor_favorita,profissao
pedro-lucas53@gma...,+7^7E%xFBc,RR,Roxo,Jogador De Golfe
rezendeisaac@hotm...,_O_2GRnGOe,GO,Ciano,Atleta De Arremes...
felipepires@uol.c...,*Aw5EOAvy9,MG,Azul,Papiloscopista
stellamoraes@bol....,mw0AWYAs#s,AC,Marrom,Aeromoça
wcarvalho@ig.com.br,pGD%!2Pq5X,AP,Laranja,Fonoaudiólogo
da-conceicaodavi-...,uhBbFxPA&9,MG,Rosa,Taxista
efreitas@bol.com.br,s#q9VZt&xl,MG,Branco,Produtor De Audio...
wnunes@bol.com.br,_8az1W%n7g,SE,Azul,Cadeirinha
jribeiro@bol.com.br,MEf1X7fj_0,PA,Marrom,Geólogo
murilo05@gmail.com,Te&gO7GkKs,MT,Marrom,Técnico De Som


In [54]:
# Aqui nós vamos utilizar o método .split para quebrar o valor da string.
# No exemplo abaixo nós utilizamos o método .split para quebrar os valor de email em 2 partes a partir do caractere "@"
# OBS: O .split aqui é igual ao do python base.
# Separamos a coluna email em nome do usuário e provedor.
# o resultado de .split sendo uma lista, para selecionar os dados basta usar o .getItem + indice da lista
# outra informação interessante foi o uso do F.col no exemplo abaixo
# perceba que, na forma da chamada da coluna df_01.email ela só funciona porque o 'email' é uma coluna nativa do 'df_01'
# caso se crie outras colunas só é possível usar essa forma de atribuição SE SALVAR essas novas colunas no dataframe
# o método F.col ignora essa regra e consegue acessar colunas criadas no DF sem a necessidade de salvá-lo

(
    df_01
    .withColumn('email_separado', F.split(df_01.email, '@'))
    .withColumn('usuario', F.split(df_01.email, '@').getItem(0))
    .withColumn('provedor', F.split(df_01.email, '@').getItem(1))
    .withColumn('nome_provedor', F.split(F.col('provedor'), '\.').getItem(0))
    .show(30,False)
    
)

+----------------------------------+----------+------+------------+------------------------------+-------------------------------------+----------------------+------------+-------------+
|email                             |senha     |estado|cor_favorita|profissao                     |email_separado                       |usuario               |provedor    |nome_provedor|
+----------------------------------+----------+------+------------+------------------------------+-------------------------------------+----------------------+------------+-------------+
|pedro-lucas53@gmail.com           |+7^7E%xFBc|RR    |Roxo        |Jogador De Golfe              |[pedro-lucas53, gmail.com]           |pedro-lucas53         |gmail.com   |gmail        |
|rezendeisaac@hotmail.com          |_O_2GRnGOe|GO    |Ciano       |Atleta De Arremesso De Peso   |[rezendeisaac, hotmail.com]          |rezendeisaac          |hotmail.com |hotmail      |
|felipepires@uol.com.br            |*Aw5EOAvy9|MG    |Azul       

<>:16: SyntaxWarning: invalid escape sequence '\.'
<>:16: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Kimura\AppData\Local\Temp\ipykernel_13788\869825367.py:16: SyntaxWarning: invalid escape sequence '\.'
  .withColumn('nome_provedor', F.split(F.col('provedor'), '\.').getItem(0))


In [64]:
# Método concat (bem padrão)

(
    df_01
    .withColumn('concat', F.concat(df_01.profissao, df_01.cor_favorita))
    .withColumn('concat_2', F.concat(df_01.estado, F.lit(' - Brasil' )))
)

email,senha,estado,cor_favorita,profissao,concat,concat_2
pedro-lucas53@gma...,+7^7E%xFBc,RR,Roxo,Jogador De Golfe,Jogador De GolfeRoxo,RR - Brasil
rezendeisaac@hotm...,_O_2GRnGOe,GO,Ciano,Atleta De Arremes...,Atleta De Arremes...,GO - Brasil
felipepires@uol.c...,*Aw5EOAvy9,MG,Azul,Papiloscopista,PapiloscopistaAzul,MG - Brasil
stellamoraes@bol....,mw0AWYAs#s,AC,Marrom,Aeromoça,AeromoçaMarrom,AC - Brasil
wcarvalho@ig.com.br,pGD%!2Pq5X,AP,Laranja,Fonoaudiólogo,FonoaudiólogoLaranja,AP - Brasil
da-conceicaodavi-...,uhBbFxPA&9,MG,Rosa,Taxista,TaxistaRosa,MG - Brasil
efreitas@bol.com.br,s#q9VZt&xl,MG,Branco,Produtor De Audio...,Produtor De Audio...,MG - Brasil
wnunes@bol.com.br,_8az1W%n7g,SE,Azul,Cadeirinha,CadeirinhaAzul,SE - Brasil
jribeiro@bol.com.br,MEf1X7fj_0,PA,Marrom,Geólogo,GeólogoMarrom,PA - Brasil
murilo05@gmail.com,Te&gO7GkKs,MT,Marrom,Técnico De Som,Técnico De SomMarrom,MT - Brasil


In [80]:
# upper | lower | initcap |  -> bem padrão
# substring basicamente é um recorte da string nas colunas, podendo indicar caractere de início e quantidade de caracteres para considerar
# format_string é igual ao .format do python


(
    df_01
    .withColumn('lower', F.lower(df.estado))
    .withColumn('upper', F.upper(df.cor_favorita))
    .withColumn('init_cap', F.initcap(F.col('lower')))
    .withColumn('substring', F.substring(df_01.cor_favorita,1,3))
    .withColumn('format_string', F.format_string('Olá %s, sua cor favorita é %s',df_01.email, df_01.cor_favorita))
    .show(10,False)
)

+----------------------------------+----------+------+------------+---------------------------+-----+-------+--------+---------+---------------------------------------------------------------+
|email                             |senha     |estado|cor_favorita|profissao                  |lower|upper  |init_cap|substring|format_string                                                  |
+----------------------------------+----------+------+------------+---------------------------+-----+-------+--------+---------+---------------------------------------------------------------+
|pedro-lucas53@gmail.com           |+7^7E%xFBc|RR    |Roxo        |Jogador De Golfe           |rr   |ROXO   |Rr      |Rox      |Olá pedro-lucas53@gmail.com, sua cor favorita é Roxo           |
|rezendeisaac@hotmail.com          |_O_2GRnGOe|GO    |Ciano       |Atleta De Arremesso De Peso|go   |CIANO  |Go      |Cia      |Olá rezendeisaac@hotmail.com, sua cor favorita é Ciano         |
|felipepires@uol.com.br            

In [116]:
# método instr mostra a posição (índice) de um str no valor da coluna
# caso seja adicionado no parâmetro de busca um valor que não exista na string ele retorna 0
# a length é padrão, conta a quantidade de caracteres na célula do texto
# repeat só repete o valor da string
# método trim elimina os espaços no início e no fim das strings
# LPAD e RPAD são métodos onde predefinimos o tamanho da string e adicionamos um valor de caractere de preenchimento

(
    df_01
    .withColumn('instr', F.instr(df_01.email, '@'))
    .withColumn('instr', F.instr(df_01.email, '@gmail'))
    .withColumn('tamanho_str', F.length(df_01.profissao))
    .withColumn('repete', F.repeat(df_01.estado, 3))
    .withColumn('Retira_Espaço', F.trim(df_01.estado))
    .withColumn('preenche_esqueda', F.lpad(df_01.cor_favorita,10,':'))
    .withColumn('preenche_direita', F.rpad(df_01.cor_favorita,10,':'))
)

email,senha,estado,cor_favorita,profissao,instr,tamanho_str,repete,Retira_Espaço,preenche_esqueda,preenche_direita
pedro-lucas53@gma...,+7^7E%xFBc,RR,Roxo,Jogador De Golfe,14,16,RRRRRR,RR,::::::Roxo,Roxo::::::
rezendeisaac@hotm...,_O_2GRnGOe,GO,Ciano,Atleta De Arremes...,0,27,GOGOGO,GO,:::::Ciano,Ciano:::::
felipepires@uol.c...,*Aw5EOAvy9,MG,Azul,Papiloscopista,0,14,MGMGMG,MG,::::::Azul,Azul::::::
stellamoraes@bol....,mw0AWYAs#s,AC,Marrom,Aeromoça,0,8,ACACAC,AC,::::Marrom,Marrom::::
wcarvalho@ig.com.br,pGD%!2Pq5X,AP,Laranja,Fonoaudiólogo,0,13,APAPAP,AP,:::Laranja,Laranja:::
da-conceicaodavi-...,uhBbFxPA&9,MG,Rosa,Taxista,0,7,MGMGMG,MG,::::::Rosa,Rosa::::::
efreitas@bol.com.br,s#q9VZt&xl,MG,Branco,Produtor De Audio...,0,24,MGMGMG,MG,::::Branco,Branco::::
wnunes@bol.com.br,_8az1W%n7g,SE,Azul,Cadeirinha,0,10,SESESE,SE,::::::Azul,Azul::::::
jribeiro@bol.com.br,MEf1X7fj_0,PA,Marrom,Geólogo,0,7,PAPAPA,PA,::::Marrom,Marrom::::
murilo05@gmail.com,Te&gO7GkKs,MT,Marrom,Técnico De Som,9,14,MTMTMT,MT,::::Marrom,Marrom::::


## Resumo

Nós testamos vários métodos de manipulação de Strings:

- `.split`: separa a string e aloca o resultado em uma lista usando como parâmetro um str
    - `.withColumn('email_separado', F.split(df_01.email, '@'))` --> caractere de separação = "@"
    - podemos acessar os valores presentes nessa lista:
        - `.withColumn('usuario', F.split(df_01.email, '@').getItem(0))` --> seleciona o resultado no índice 0
        - `.withColumn('provedor', F.split(df_01.email, '@').getItem(1))` --> seleciona o resultado no índice 1
    - Podemos realizar o `.split` usando alguma coluna criada no próprio código, porém devemos usar o método `.col`:
        - `.withColumn('nome_provedor', F.split(F.col('provedor'), '\.').getItem(0))` --> selecionei o índice 0

- `.concat`: junta dois trechos de STR
    - `.withColumn('concat', F.concat(df_01.profissao, df_01.cor_favorita))` --> concatenando valores de 2 colunas;
    - `.withColumn('concat_2', F.concat(df_01.estado, F.lit(' - Brasil' )))` --> concatenando o valor de uma coluna com um trecho str literal com o `.lit`

- LOWER -> todos caracteres minusculas: `.withColumn('lower', F.lower(df.estado))`
- UPPER -> todos caracteres maiúsculas: `.withColumn('upper', F.upper(df.cor_favorita))`
- INIT_CAP -> primeira letra maiúscula: `.withColumn('init_cap', F.initcap(F.col('lower')))`

- substring: possibilita um recorte dos valores da string
    - parâmetros: coluna, string inicial, quantidade de strings
    - `.withColumn('substring', F.substring(df_01.cor_favorita,1,3))`

- format_string: é IGUAL ao `.format` do python, única diferençã é que utilizamos essa notação com o `%d`,`%s`,`%i`, etc...:
    - `.withColumn('format_string', F.format_string('Olá %s, sua cor favorita é %s',df_01.email, df_01.cor_favorita))`

- instr: mostra o índice que o caractere se encontra:
    - `.withColumn('instr', F.instr(df_01.email, '@'))` --> str base para a busca é o '@' (segundo parâmetro)
    - Caso não exista o caractere no texto em questão, retorno é 0

- length: tamanha da string:
    - `.withColumn('tamanho_str', F.length(df_01.profissao))` --> Motorista = 9 caracteres
 
- repeat: repete o valor da string n vezes 
    - `.withColumn('repete', F.repeat(df_01.estado, 3))` --> nesse caso vai repetir 3 vezes

- trim: elimina os espaços iniciais e finais na string:
    - `.withColumn('Retira_Espaço', F.trim(df_01.estado))` --> "....olá......" ==>> "olá"

- lpad & rpad: determina um valor fixo para a string, caso o valor importado não preencha por completo, ele completa com algum outro caractere:
    - `.withColumn('preenche_esqueda', F.lpad(df_01.cor_favorita,10,':'))` - n=10 | str="Roxo" | caractere=":" ==>> ::::::Roxo	lpad (LADO ESQUERDO)
    - `.withColumn('preenche_direita', F.rpad(df_01.cor_favorita,10,':'))` - n=10 | str="Roxo" | caractere=":" ==>> Roxo::::::   rpad (LADO DIREITO)

